![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_MPNetForQuestionAnswering.ipynb)

# Import ONNX MPNet models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- The MPNetForQuestionAnswering model was introduced in `Spark NLP 5.2.4`

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.35.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx] optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use the [haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all](https://huggingface.co/haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

! optimum-cli export onnx --model {MODEL_NAME} {EXPORT_PATH}

2025-04-30 16:29:05.693376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746030546.051025    1491 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746030546.141925    1491 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 16:29:06.888556: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
config.json: 100% 637/637 [00:00<00:00, 2.95MB/s]
pytorch_model.bin: 100% 436M/436M [00:06<00:00, 67.7MB/s]
tokenizer_config.

We have to move additional model assets (tokenizer vocabulary and configs) into a separate folder, so that Spark NLP can load it properly.

In [3]:
! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/*.json {EXPORT_PATH}/*.txt

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 425656
drwxr-xr-x 2 root root      4096 Apr 30 16:30 assets
-rw-r--r-- 1 root root 435859895 Apr 30 16:29 model.onnx


In [5]:
!ls -l {EXPORT_PATH}/assets

total 936
-rw-r--r-- 1 root root    606 Apr 30 16:29 config.json
-rw-r--r-- 1 root root    964 Apr 30 16:29 special_tokens_map.json
-rw-r--r-- 1 root root   1614 Apr 30 16:29 tokenizer_config.json
-rw-r--r-- 1 root root 710944 Apr 30 16:29 tokenizer.json
-rw-r--r-- 1 root root 231536 Apr 30 16:29 vocab.txt


## Import and Save MPNet in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [6]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Mounted at /content/drive
Processing ./spark_nlp-6.0.0-py2.py3-none-any.whl
Apache Spark version: 3.5.5


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `MPNetForQuestionAnswering` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `MPNetForQuestionAnswering` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import *

# All these params should be identical to the original ONNX model
question_answering = (
    MPNetForQuestionAnswering.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("document_question", "document_context")
    .setOutputCol("answer")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
question_answering.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX MPNet model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 425724
drwxr-xr-x 3 root root      4096 Apr 30 16:31 fields
drwxr-xr-x 2 root root      4096 Apr 30 16:31 metadata
-rw-r--r-- 1 root root 435926539 Apr 30 16:31 mpnet_question_answering_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny MPNet model 😊

In [12]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

question_answering = MPNetForQuestionAnswering.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer") \
    .setCaseSensitive(False)

pipeline = Pipeline().setStages([
    document_assembler,
    question_answering
])
data = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(data).transform(data)
result.select("answer.result").show(truncate=False)

+-------+
|result |
+-------+
|[Clara]|
+-------+



That's it! You can now go wild and use hundreds of MPNet models from HuggingFace 🤗 in Spark NLP 🚀
